<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/text_translation_wrapper_rmv20230215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m spacy download fr_core_news_md

!pip install transformers[sentencepiece]




In [ ]:
import pandas as pd
import os
import spacy

import fr_core_news_md

from transformers import pipeline

In [ ]:
spacy_model = fr_core_news_md.load()
translation_model = pipeline("text2text-generation", model='Helsinki-NLP/opus-mt-fr-en')

In [1]:
def text_translation(input_file_path, output_folder_path):
    

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path )

    if os.path.exists(input_file_path):
        print("Text extraction process starting for...",input_file_path)
        
        file_name = os.path.basename(input_file_path).split('.')[0]
        print('File_name: ', file_name)

        text_file_name = output_folder_path + file_name + ".txt"
        print('tex_file_name: ', text_file_name)

        with open(input_file_path,encoding="utf8") as f:
            txt=f.read()

        doc = spacy_model(txt)
        translated_string = ''
        
        for sentence in doc.sents:
            print(sentence.text)
            print(translation_model(sentence.text), '\n')
            print("End of sentence...", '\n')
            translated_string = translated_string + '\n' + translation_model(sentence.text)[0]['generated_text']
            
            with open(text_file_name, 'w',encoding="utf8") as f:
                f.write(translated_string)
    else:
        raise ValueError("File does not exist!!")

    return text_file_name

In [2]:
def text_translation_wrapper(metadata_file_path, output_folder_path):
    
    metadata = pd.read_csv(metadata_file_path)
    # print(metadata.head())
    print(metadata.columns)

    # # ## When metadata file does not have any column by the name 'doc_text_file_path', we will proceed with all the files 
    if 'translated_text_file_path' not in metadata.columns:
        print("translated_text_file_path not present in metadata...proceeding with all the files")

        metadata.loc[:,'translated_text_file_path'] = metadata["text_file_path"].apply(lambda x: text_translation(x, output_folder_path))
    
    # When metadata file has column by the name 'doc_text_file_path', we with process with only those files for which we dont have value in doc_text_file_path 
    else:
        print("text_file_path present in metadata...proceeding with only those files for which text_file_path is missing")
        metadata_fil = metadata[(metadata['text_file_path']=='') | (metadata['text_file_path'].isnull())]
        metadata_fil.loc[:,'translated_text_file_path'] = metadata_fil["text_file_path"].apply(lambda x: text_translation(x, output_folder_path))
        metadata = pd.concat([metadata[(metadata['translated_text_file_path']!='') & (metadata['translated_text_file_path'].notnull())], metadata_fil], ignore_index=True)

    metadata.to_csv(metadata_file_path, index=False)


In [ ]:
metadata_file_path = r"/content/drive/MyDrive/Zenon_POC_OCR/metadata.csv"
output_folder_path = "/content/drive/MyDrive/Zenon_POC_OCR/2_output/"
text_translation(metadata_file_path, output_folder_path)
